In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Reading the files

In [12]:
## Reading the files

def read_inventory_files(folder_path, sheet_pattern, columns_to_extract, output_filename):
    """
    Read all rows properly from lead service line inventory Excel files
    
    Args:
        folder_path: Path to folder containing Excel files
        sheet_pattern: Text pattern to identify the correct sheet
        columns_to_extract: List of column names to extract
        output_filename: Name for the output file
    """
    print(f"Processing {sheet_pattern} data...")
    all_data = []
    
    for file_name in os.listdir(folder_path):
        if not file_name.endswith(('.xlsx', '.xls')) or file_name.startswith('~$'):
            continue
            
        file_path = os.path.join(folder_path, file_name)
        print(f"Reading: {file_name}")
        
        try:
            # Get all sheet names
            xl = pd.ExcelFile(file_path, engine="openpyxl")
            
            # Find sheet that matches pattern
            sheet_name = None
            for sheet in xl.sheet_names:
                if sheet_pattern.lower() in sheet.lower():
                    sheet_name = sheet
                    break
            
            if not sheet_name:
                print(f"  - No matching sheet in {file_name}")
                continue
            
            # First, try to find the header row
            # Read more rows to locate headers
            sample = pd.read_excel(file_path, sheet_name=sheet_name, nrows=30, header=None)

            # Try to find a row containing our key columns
            header_row = None
            header_scores = []

            for i in range(min(30, len(sample))):
                row = sample.iloc[i].astype(str).str.upper()
                # Check if this row contains several of our column names
                matches = 0
                keywords = ["SITE", "ID", "LOCATION", "ADDRESS", "MATERIAL", "STREET", "TOWN"]
                
                for keyword in keywords:
                    if any(keyword in str(val) for val in row):
                        matches += 1
                        
                header_scores.append((i, matches))
                
                # If we find enough matches, use this as header
                if matches >= 3:
                    header_row = i
                    break

            # If we didn't find a clear winner, use the row with highest score
            if header_row is None and header_scores:
                header_scores.sort(key=lambda x: x[1], reverse=True)
                if header_scores[0][1] >= 2:  # At least 2 matches
                    header_row = header_scores[0][0]
                    print(f"  - Using best header candidate at row {header_row} with score {header_scores[0][1]}")
            
            # If we found a header row, use it
            if header_row is not None:
                # Now read the full data with correct header
                df = pd.read_excel(file_path, sheet_name=sheet_name, header=header_row)
                
                # Standardize column names (uppercase)
                df.columns = df.columns.str.upper().str.strip()
                
                # IMPORTANT: Fix for numeric/unnamed columns
                # Replace any numeric column names with descriptive names
                numeric_cols = [col for col in df.columns if isinstance(col, (int, float)) or str(col).isdigit()]
                for col in numeric_cols:
                    df = df.rename(columns={col: f"UNNAMED_{col}"})
                
                # Debug: print all available columns
                print(f"  - Available columns: {df.columns.tolist()}")

                # NEW: Check if we need to combine STREET NUMBER and STREET NAME into STREET ADDRESS
                has_street_number = any("STREET NUMBER" in str(col).upper() for col in df.columns)
                has_street_name = any("STREET NAME" in str(col).upper() for col in df.columns)
                has_street_address = any("STREET ADDRESS" in str(col).upper() for col in df.columns)
                
                # Only combine if we have number+name but no address
                if has_street_number and has_street_name and not has_street_address:
                    print(f"  - File has separate STREET NUMBER and STREET NAME columns, will combine into STREET ADDRESS")
                    
                    # Find the exact column names for STREET NUMBER and STREET NAME
                    street_number_col = next((col for col in df.columns if "STREET NUMBER" in str(col).upper()), None)
                    street_name_col = next((col for col in df.columns if "STREET NAME" in str(col).upper()), None)
                    
                    if street_number_col and street_name_col:
                        # Convert to string and handle NaN values
                        df[street_number_col] = df[street_number_col].fillna('').astype(str)
                        df[street_name_col] = df[street_name_col].fillna('').astype(str) 
                        
                        # Combine the columns into a new STREET ADDRESS column
                        df["STREET ADDRESS"] = df[street_number_col] + " " + df[street_name_col]
                        # Clean up any double spaces
                        df["STREET ADDRESS"] = df["STREET ADDRESS"].str.replace("  ", " ").str.strip()
                        print(f"  - Created STREET ADDRESS column by combining {street_number_col} and {street_name_col}")
                
                # Prepare to filter and map columns
                cols_to_keep = []
                col_mapping = {}
                
                # MODIFIED: Keep track of which target columns we've already matched
                # This prevents multiple columns mapping to the same target
                matched_targets = set()

                for col in df.columns:
                    # Skip unnamed columns completely
                    if "UNNAMED" in str(col).upper() or str(col).isdigit():
                        continue
                        
                    # Check if this column approximately matches any of our target columns
                    for target_col in columns_to_extract:
                        # Skip targets we've already matched
                        if target_col in matched_targets:
                            continue
                            
                        # SPECIAL CASE: If we created a STREET ADDRESS column and this target is STREET ADDRESS,
                        # we should only match our newly created column
                        if target_col == "STREET ADDRESS" and "STREET ADDRESS" in df.columns:
                            if col == "STREET ADDRESS":
                                print(f"  - Using newly created '{col}' for target '{target_col}'")
                                cols_to_keep.append(col)
                                col_mapping[col] = target_col
                                matched_targets.add(target_col)
                            # Skip all other potential matches for STREET ADDRESS
                            continue
                        
                        # If target is STREET NUMBER or STREET NAME but we've already handled STREET ADDRESS,
                        # don't match these separately
                        if (target_col == "STREET NUMBER" or target_col == "STREET NAME") and "STREET ADDRESS" in matched_targets:
                            continue
                        
                        # Create variations of target column names to improve matching
                        target_variations = [
                            target_col,
                            target_col.upper(),
                            target_col.replace(" ", ""),
                            target_col.replace("IDENTIFIER", "ID"),
                            target_col.replace("ADDRESS", "ADDR"),
                            # Common abbreviations for address-related fields
                            "STREET" if target_col == "STREET ADDRESS" else None,
                            "ST NUM" if target_col == "STREET NUMBER" else None,
                            "ST NAME" if target_col == "STREET NAME" else None,
                            "LOC ID" if target_col == "LOCATION IDENTIFIER" else None
                        ]
                        target_variations = [v for v in target_variations if v]  # Remove None values
                        
                        # Check for various matching patterns
                        matched = False
                        
                        # Check exact match
                        if col == target_col or col == target_col.upper():
                            matched = True
                        # Check if column contains target or vice versa
                        elif any(var in col for var in target_variations) or any(col in var for var in target_variations):
                            matched = True
                        # Check normalized versions (no spaces)
                        elif any(var.replace(" ", "") == col.replace(" ", "") for var in target_variations):
                            matched = True
                            
                        if matched:
                            print(f"  - Matched column '{col}' to target '{target_col}'")
                            cols_to_keep.append(col)
                            col_mapping[col] = target_col
                            matched_targets.add(target_col)
                            break

                # If important columns are missing, look again with more flexible matching
                key_columns = ["SITE ID", "LOCATION IDENTIFIER", "STREET ADDRESS"]
                missing_keys = [col for col in key_columns if col not in matched_targets]

                if missing_keys:
                    print(f"  - Missing key columns: {missing_keys}, trying more flexible matching")
                    
                    for missing_col in missing_keys:
                        # Define more specific keywords for each missing column
                        if missing_col == "SITE ID":
                            keywords = ["SITE", "ID", "FACILITY", "FAC ID", "FACILITY ID", "BUILDING ID"]
                        elif missing_col == "LOCATION IDENTIFIER":
                            keywords = ["LOCATION", "LOC", "POSITION", "PLACE", "IDENT"]
                        elif missing_col == "STREET ADDRESS":
                            keywords = ["ADDRESS", "ADDR", "LOCATION", "STREET", "ST ADDR", "PROPERTY"]
                            
                        # Check all available columns again
                        for col in df.columns:
                            # Skip already matched columns
                            if col in cols_to_keep:
                                continue
                                
                            if any(keyword.upper() in col.upper() for keyword in keywords):
                                print(f"  - Flexible match: '{col}' to '{missing_col}'")
                                cols_to_keep.append(col)
                                col_mapping[col] = missing_col
                                matched_targets.add(missing_col)
                                break
                
                if cols_to_keep:
                    # Keep only matched columns
                    df = df[cols_to_keep].copy()
                    
                    # Rename to standard names
                    df = df.rename(columns=col_mapping)
                    
                    # Add source file column
                    df["SOURCE FILE"] = file_name
                    
                    # Drop rows where all values (except SOURCE FILE) are missing
                    df = df.dropna(subset=list(set(col_mapping.values())), how='all')
                    
                    # Handle header rows that might have been read as data
                    # If a row has text that matches column names, it's likely a header row
                    headers_in_data = []
                    for i, row in df.iterrows():
                        row_values = row.astype(str).str.upper()
                        if sum(any(col.upper() in val for val in row_values) 
                              for col in ["SITE ID", "LOCATION", "ADDRESS", "MATERIAL"]) >= 2:
                            headers_in_data.append(i)
                    
                    # Drop identified header rows from data
                    if headers_in_data:
                        df = df.drop(headers_in_data)
                    
                    # Debug: check how many non-empty values we have for key columns
                    for key_col in ["SITE ID", "LOCATION IDENTIFIER", "STREET ADDRESS"]:
                        if key_col in df.columns:
                            non_empty = df[key_col].astype(str).str.strip().str.len() > 0
                            print(f"  - {key_col}: {non_empty.sum()} non-empty values out of {len(df)}")
                    
                    # Append to our collection
                    all_data.append(df)
                    print(f"  - Added {len(df)} rows")
                else:
                    print(f"  - No matching columns found in {file_name}")
                
            else:
                print(f"  - Could not identify header row in {file_name}")
                
        except Exception as e:
            print(f"  - Error processing {file_name}: {e}")
            # Print more detailed error information
            import traceback
            print(f"  - Error details: {traceback.format_exc()}")
    
    # Combine all data
    if all_data:
        # Combine all dataframes
        print(f"Combining data from {len(all_data)} files...")
        
        # Ensure all dataframes have the same column structure before combining
        all_columns = []
        for df in all_data:
            all_columns.extend(df.columns.tolist())
        
        unique_columns = list(dict.fromkeys([col for col in all_columns 
                                            if col in columns_to_extract or col == "SOURCE FILE"]))
        
        for i in range(len(all_data)):
            # Add missing columns
            for col in unique_columns:
                if col not in all_data[i].columns:
                    all_data[i][col] = None
                    
            # Ensure dataframe only has our expected columns
            all_data[i] = all_data[i][unique_columns]
        
        # Combine and save
        final_df = pd.concat(all_data, ignore_index=True)
        
        # Replace any "nan" or numeric values in string columns with empty strings
        for col in final_df.columns:
            # Skip SOURCE FILE column
            if col == "SOURCE FILE":
                continue
                
            # For string columns, clean up nan values
            if final_df[col].dtype == 'object':
                # Replace nan, NULL, None etc with empty string
                final_df[col] = final_df[col].astype(str)
                final_df[col] = final_df[col].replace({"nan": "", "None": "", "NaN": "", "NULL": "", "null": ""})
                
                # MODIFIED: Only clean nan values in SITE ID, but keep numeric values
                if col == "SITE ID":
                    final_df[col] = final_df[col].apply(
                        lambda x: "" if x in ["nan", "None", "NaN", "NULL", "null"] else x
                    )
                    # This replaces the original code that emptied numeric IDs:
                    # lambda x: "" if str(x).isdigit() or x in ["0", "nan", "None"] else x
        
        # Drop completely empty rows
        final_df = final_df.loc[~final_df.drop(columns=["SOURCE FILE"]).apply(
            lambda x: all(val == "" for val in x), axis=1
        )]
        
        # Save to Excel
        output_path = os.path.join(folder_path, output_filename)
        final_df.to_excel(output_path, index=False, engine="openpyxl")
        print(f"Saved {len(final_df)} rows to {output_path}")
        
        return final_df
    else:
        print("No data found")
        return None


## Testing part 1

# STEP 1: Process Initial Material Inventory

In [3]:
# Define the folder paths
folder_paths = ["testing-part1"]  # Make sure your Excel file is in this folder

def step_1_process_initial_inventory():
    """Step 1: Process Initial Material Inventory files"""
    print("\n" + "="*50)
    print("STEP 1: PROCESSING INITIAL MATERIAL INVENTORY")
    print("="*50)
    
    # Define columns to extract for initial inventory
    initial_columns = [
        "SITE ID",
        "LOCATION IDENTIFIER",
        "STREET ADDRESS",
        "STREET NAME",        
        "STREET NUMBER",      
        "TOWN",
        "STATE",
        "ENTIRE SERVICE LINE MATERIAL CLASSIFICATION",
        "BUILDING TYPE"
    ]
    
    all_dfs = []
    # Process initial inventory files for each folder path
    for folder_path in folder_paths:
        print(f"\nProcessing folder: {folder_path}")
        initial_df = read_inventory_files(
            folder_path=folder_path,
            sheet_pattern="Initial Material Inventory",  # MODIFIED: Matched exact case in the Excel file
            columns_to_extract=initial_columns,
            output_filename=f"{folder_path}_initial_material.xlsx"
        )
        if initial_df is not None:
            all_dfs.append(initial_df)
    
    # Combine data from both folders if needed
    if len(all_dfs) > 1:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        output_path = "combined_initial_material.xlsx"
        combined_df.to_excel(output_path, index=False, engine="openpyxl")
        print(f"Saved combined {len(combined_df)} rows to {output_path}")
        final_df = combined_df
    elif len(all_dfs) == 1:
        final_df = all_dfs[0]
    else:
        final_df = None
    
    # Print summary of key columns to verify data quality
    if final_df is not None:
        print("\nData Quality Summary:")
        total_rows = len(final_df)
        for col in ["SITE ID", "LOCATION IDENTIFIER", "STREET ADDRESS", "STREET NAME", "STREET NUMBER"]:
            if col in final_df.columns:
                non_empty = final_df[col].astype(str).str.strip().str.len() > 0
                count = non_empty.sum()
                percent = (count / total_rows * 100) if total_rows > 0 else 0
                print(f"  - {col}: {count}/{total_rows} non-empty values ({percent:.1f}%)")
    
    print("Initial Material Inventory processing complete!")
    return final_df

## Running step

In [4]:
# Execute Step 1
if __name__ == "__main__":
    # Run Step 1 only
    initial_df = step_1_process_initial_inventory()
    print("\nStep 1 processing complete!")


STEP 1: PROCESSING INITIAL MATERIAL INVENTORY

Processing folder: testing-part1
Processing Initial Material Inventory data...
Reading: CT0780121_LCRR_Inventory_Initial_10.16.24.xlsx
  - Available columns: ['SITE ID', 'LOCATION IDENTIFIER', 'STREET ADDRESS', 'TOWN', 'STATE', 'DISADVANTAGED NEIGHBORHOOD?', 'CONNECTOR OR GOOSENECK MATERIAL CODE', 'CONNECTOR OR GOOSENECK MATERIAL', 'CONNECTOR OR GOOSENECK MATERIAL INSTALL YEAR', 'WAS CONNECTOR OR GOOSENECK PREVIOUSLY LEAD?', 'MAIN SIDE OF CURB STOP SERVICE LINE OWNERSHIP', 'MAIN SIDE OF CURB STOP SERVICE LINE MATERIAL CODE\n(INITIAL INVENTORY ONLY)', 'MAIN SIDE OF CURB STOP SERVICE LINE MATERIAL', 'WAS MAIN SIDE OF CURB STOP SERVICE LINE MATERIAL EVER PREVIOUSLY LEAD?', 'MAIN SIDE OF CURB STOP SERVICE LINE DIAMETER (INCHES)', 'MAIN SIDE OF CURB STOP SERVICE LINE INSTALL YEAR', 'MAIN SIDE OF CURB STOP SERVICE LINE VERIFICATION DATE', 'MAIN SIDE OF CURB STOP SERVICE LINE VERIFICATION SOURCE', 'IS MAIN SIDE OF CURB STOP SERVICE LINE REPLACEM

## Testing part 2

# STEP 1: Process Initial Material Inventory

In [5]:
# Define the folder paths
folder_paths = ["testing-part2"]  # Make sure your Excel file is in this folder

def step_1_process_initial_inventory():
    """Step 1: Process Initial Material Inventory files"""
    print("\n" + "="*50)
    print("STEP 1: PROCESSING INITIAL MATERIAL INVENTORY")
    print("="*50)
    
    # Define columns to extract for initial inventory
    initial_columns = [
        "SITE ID",
        "LOCATION IDENTIFIER",
        "STREET ADDRESS",
        "STREET NAME",        
        "STREET NUMBER",      
        "TOWN",
        "STATE",
        "ENTIRE SERVICE LINE MATERIAL CLASSIFICATION",
        "BUILDING TYPE"
    ]
    
    all_dfs = []
    # Process initial inventory files for each folder path
    for folder_path in folder_paths:
        print(f"\nProcessing folder: {folder_path}")
        initial_df = read_inventory_files(
            folder_path=folder_path,
            sheet_pattern="Initial Material Inventory",  # MODIFIED: Matched exact case in the Excel file
            columns_to_extract=initial_columns,
            output_filename=f"{folder_path}_initial_material.xlsx"
        )
        if initial_df is not None:
            all_dfs.append(initial_df)
    
    # Combine data from both folders if needed
    if len(all_dfs) > 1:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        output_path = "combined_initial_material.xlsx"
        combined_df.to_excel(output_path, index=False, engine="openpyxl")
        print(f"Saved combined {len(combined_df)} rows to {output_path}")
        final_df = combined_df
    elif len(all_dfs) == 1:
        final_df = all_dfs[0]
    else:
        final_df = None
    
    # Print summary of key columns to verify data quality
    if final_df is not None:
        print("\nData Quality Summary:")
        total_rows = len(final_df)
        for col in ["SITE ID", "LOCATION IDENTIFIER", "STREET ADDRESS", "STREET NAME", "STREET NUMBER"]:
            if col in final_df.columns:
                non_empty = final_df[col].astype(str).str.strip().str.len() > 0
                count = non_empty.sum()
                percent = (count / total_rows * 100) if total_rows > 0 else 0
                print(f"  - {col}: {count}/{total_rows} non-empty values ({percent:.1f}%)")
    
    print("Initial Material Inventory processing complete!")
    return final_df

## Running step

In [6]:
# Execute Step 1
if __name__ == "__main__":
    # Run Step 1 only
    initial_df = step_1_process_initial_inventory()
    print("\nStep 1 processing complete!")


STEP 1: PROCESSING INITIAL MATERIAL INVENTORY

Processing folder: testing-part2
Processing Initial Material Inventory data...
Reading: CT0473011_LCRR_Inventory_Initial_10.16.24.xlsx
  - Available columns: ['SITE ID', 'LOCATION IDENTIFIER', 'STREET ADDRESS', 'TOWN', 'STATE', 'DISADVANTAGED NEIGHBORHOOD?', 'CONNECTOR OR GOOSENECK MATERIAL CODE', 'CONNECTOR OR GOOSENECK MATERIAL', 'CONNECTOR OR GOOSENECK MATERIAL INSTALL YEAR', 'WAS CONNECTOR OR GOOSENECK PREVIOUSLY LEAD?', 'MAIN SIDE OF CURB STOP SERVICE LINE OWNERSHIP', 'MAIN SIDE OF CURB STOP SERVICE LINE MATERIAL CODE\n(INITIAL INVENTORY ONLY)', 'MAIN SIDE OF CURB STOP SERVICE LINE MATERIAL', 'WAS MAIN SIDE OF CURB STOP SERVICE LINE MATERIAL EVER PREVIOUSLY LEAD?', 'MAIN SIDE OF CURB STOP SERVICE LINE DIAMETER (INCHES)', 'MAIN SIDE OF CURB STOP SERVICE LINE INSTALL YEAR', 'MAIN SIDE OF CURB STOP SERVICE LINE VERIFICATION DATE', 'MAIN SIDE OF CURB STOP SERVICE LINE VERIFICATION SOURCE', 'IS MAIN SIDE OF CURB STOP SERVICE LINE REPLACEM

## double checking CT0150011_LCRR_Inventory_Initial_10.16.24.xlsx

In [13]:
# Define the folder paths
folder_paths = ["doublecheck"]  # Make sure your Excel file is in this folder

def step_1_process_initial_inventory():
    """Step 1: Process Initial Material Inventory files"""
    print("\n" + "="*50)
    print("STEP 1: PROCESSING INITIAL MATERIAL INVENTORY")
    print("="*50)
    
    # Define columns to extract for initial inventory
    initial_columns = [
        "SITE ID",
        "LOCATION IDENTIFIER",
        "STREET ADDRESS",
        "STREET NAME",        
        "STREET NUMBER",      
        "TOWN",
        "STATE",
        "ENTIRE SERVICE LINE MATERIAL CLASSIFICATION",
        "BUILDING TYPE"
    ]
    
    all_dfs = []
    # Process initial inventory files for each folder path
    for folder_path in folder_paths:
        print(f"\nProcessing folder: {folder_path}")
        initial_df = read_inventory_files(
            folder_path=folder_path,
            sheet_pattern="Initial Material Inventory",  # MODIFIED: Matched exact case in the Excel file
            columns_to_extract=initial_columns,
            output_filename=f"{folder_path}_initial_material.xlsx"
        )
        if initial_df is not None:
            all_dfs.append(initial_df)
    
    # Combine data from both folders if needed
    if len(all_dfs) > 1:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        output_path = "combined_initial_material.xlsx"
        combined_df.to_excel(output_path, index=False, engine="openpyxl")
        print(f"Saved combined {len(combined_df)} rows to {output_path}")
        final_df = combined_df
    elif len(all_dfs) == 1:
        final_df = all_dfs[0]
    else:
        final_df = None
    
    # Print summary of key columns to verify data quality
    if final_df is not None:
        print("\nData Quality Summary:")
        total_rows = len(final_df)
        for col in ["SITE ID", "LOCATION IDENTIFIER", "STREET ADDRESS", "STREET NAME", "STREET NUMBER"]:
            if col in final_df.columns:
                non_empty = final_df[col].astype(str).str.strip().str.len() > 0
                count = non_empty.sum()
                percent = (count / total_rows * 100) if total_rows > 0 else 0
                print(f"  - {col}: {count}/{total_rows} non-empty values ({percent:.1f}%)")
    
    print("Initial Material Inventory processing complete!")
    return final_df

In [14]:
# Execute Step 1
if __name__ == "__main__":
    # Run Step 1 only
    initial_df = step_1_process_initial_inventory()
    print("\nStep 1 processing complete!")


STEP 1: PROCESSING INITIAL MATERIAL INVENTORY

Processing folder: doublecheck
Processing Initial Material Inventory data...
Reading: CT0150011_LCRR_Inventory_Initial_10.16.24 copy.xlsx
  - Available columns: ['SITE ID', 'LOCATION IDENTIFIER', 'STREET ADDRESS', 'TOWN', 'STATE', 'DISADVANTAGED NEIGHBORHOOD?', 'CONNECTOR OR GOOSENECK MATERIAL CODE', 'CONNECTOR OR GOOSENECK MATERIAL', 'CONNECTOR OR GOOSENECK MATERIAL INSTALL YEAR', 'WAS CONNECTOR OR GOOSENECK PREVIOUSLY LEAD?', 'MAIN SIDE OF CURB STOP SERVICE LINE OWNERSHIP', 'MAIN SIDE OF CURB STOP SERVICE LINE MATERIAL CODE\n(INITIAL INVENTORY ONLY)', 'MAIN SIDE OF CURB STOP SERVICE LINE MATERIAL', 'WAS MAIN SIDE OF CURB STOP SERVICE LINE MATERIAL EVER PREVIOUSLY LEAD?', 'MAIN SIDE OF CURB STOP SERVICE LINE DIAMETER (INCHES)', 'MAIN SIDE OF CURB STOP SERVICE LINE INSTALL YEAR', 'MAIN SIDE OF CURB STOP SERVICE LINE VERIFICATION DATE', 'MAIN SIDE OF CURB STOP SERVICE LINE VERIFICATION SOURCE', 'IS MAIN SIDE OF CURB STOP SERVICE LINE REPLA